Functions that used with cloud Files

## cloudFiles.schemaLocation
- Stores inferred schema
- Tracks schema evolution
- Makes pipeline restart-safe

In [0]:
map(
    'cloudFiles.schemaLocation',
    '/mnt/schema/customers'
  )

## cloudFiles.inferColumnTypes
- Infers data types instead of string

In [0]:
map(
    'cloudFiles.inferColumnTypes','true'
  )

## cloudFiles.schemaEvolutionMode
- none (fail on schema change)
- addNewColumns (MOST COMMON)
- c) rescue (SAFEST)

In [0]:
 map(
    'cloudFiles.schemaEvolutionMode','none'
  )

In [0]:
 map(
    'cloudFiles.schemaEvolutionMode','addNewColumns'
  )

In [0]:
map(
    'cloudFiles.schemaEvolutionMode','rescue',
    'cloudFiles.rescuedDataColumn','_rescued_data'
  )

## cloudFiles.rescuedDataColumn
- Stores unexpected fields as JSON

## cloudFiles.includeExistingFiles
- Loads historical files once
- Then processes only new files

In [0]:
map(
    'cloudFiles.includeExistingFiles','true'
  )

## cloudFiles.useNotifications
- Uses cloud events instead of file listing
- Faster and cheaper
- cloudFiles.useNotifications is an Auto Loader option that tells Apache Spark Auto Loader to use cloud-native event notifications instead of directory listing to detect new files.

- ❌ Without notifications (default listing)
- Auto Loader:
- Lists files repeatedly
- Scans directories
- Slows down as file count grows
- Costs more on cloud storage APIs
- 
- ✅ With notifications
- Auto Loader:
- Listens to file arrival events
- Knows exactly which files arrived
- Avoids expensive directory scans
- Scales to millions of files

In [0]:
 map(
    'cloudFiles.useNotifications','true'
  )

## cloudFiles.maxFilesPerTrigger
- Controls ingestion rate
- Max 50 files per micro-batch.

In [0]:
 map(
    'cloudFiles.maxFilesPerTrigger','50'
  )